<a href="https://colab.research.google.com/github/DvAzevedo/Hybrid_Recommendation_System/blob/main/movies_mapping_dbpedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!pip install bs4
!pip install requests

import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
PROJ_ROOT = "/content/drive/MyDrive/SemanticRec"
DATA_DIR  = f"{PROJ_ROOT}/Data"
DATA_PATH        = '/content/drive/MyDrive/SemanticRec/Data/movies_amostra_director.csv' #coloque o caminho correto
OUT_PATH =  '/content/drive/MyDrive/SemanticRec/Data/movies_amostra_url.csv' #coloque o caminho correto
df = pd.read_csv(DATA_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install --quiet SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

# Inicializa conexão com DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
# Função para consultar DBpedia
def buscar_url_dbpedia(titulo, diretor):
    if pd.isna(titulo) or pd.isna(diretor):
        return ""

    query = f"""
    SELECT ?film WHERE {{
      ?film a dbo:Film ;
            rdfs:label ?filmName ;
            dbo:director ?director .
      ?director rdfs:label ?directorName .
      FILTER (LANG(?filmName) = 'en')
      FILTER (LANG(?directorName) = 'en')
      FILTER (CONTAINS(LCASE(?filmName), LCASE("{titulo}")))
      FILTER (CONTAINS(LCASE(?directorName), LCASE("{diretor}")))
    }}
    LIMIT 1
    """
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        for result in results["results"]["bindings"]:
            return result["film"]["value"]
        return ""
    except:
        return ""

In [21]:
if 'dbpediaURL' not in df.columns:
    df['dbpediaURL'] = ""

# Rodar apenas para linhas que ainda não têm dbpediaURL e têm título + diretor
for i, row in df.iterrows():
    if row.get("dbpediaURL", "") or pd.isna(row["Titulo"]) or pd.isna(row["Diretor"]):
        continue

    titulo = str(row["Titulo"]).strip()
    diretor = str(row["Diretor"]).strip()
    url = buscar_url_dbpedia(titulo, diretor)
    df.at[i, 'dbpediaURL'] = url
    print(f"{i}: {titulo} → {diretor} → {url}")
    time.sleep(1)
    if i > 100:
        break

0: Toy Story → John Lasseter → http://dbpedia.org/resource/Toy_Story_2
1: Jumanji → Joe Johnston → http://dbpedia.org/resource/Jumanji
2: Grumpier Old Men → Howard Deutch → http://dbpedia.org/resource/Grumpier_Old_Men


In [23]:
df.to_csv('/content/drive/MyDrive/SemanticRec/Data/movies_amostra_url.csv', index=False)